In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Load libraries
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import metrics

pd.set_option("display.max_columns", None)

In [3]:
# load dataset
data = pd.read_json('/content/drive/MyDrive/Accountstory/data/TRAINING_dataset.json')
data

,gender,stakeholder_category,us_name,uni_rank,degree_score,edu_marketing,edu_communication,edu_english,edu_political science,edu_business management,edu_psychology,edu_liberal studies,edu_communications,edu_german,edu_electrical engineering,edu_electrical engineering and computer science,edu_computer science,edu_software engineering,edu_management,edu_education,edu_organizational leadership,edu_business administration,edu_business administration and management,edu_business,edu_media studies,edu_entrepreneurship,edu_computer engineering,edu_broadcasting,edu_human resources,edu_biochemistry,edu_accounting,edu_broadcast journalism,edu_mathematics,edu_liberal arts,edu_history,edu_art,edu_data science,edu_applied physics,edu_physics,edu_information systems,edu_design,edu_leadership,edu_international relations,edu_arabic,edu_entrepreneurial studies,edu_computer and information systems,edu_drawing,edu_electronics,edu_nanotechnology,edu_engineering,edu_economics,edu_business marketing,edu_sport management,edu_architecture,edu_accountancy,edu_french language,edu_literature,edu_french language and literature,edu_english language and literature,edu_english language,edu_graphic design,edu_art history,edu_anthropology,edu_politics,edu_russian,edu_international studies,edu_real estate,edu_finance,edu_international affairs,edu_sociology,edu_teaching,edu_business economics,edu_medicine,edu_biology,edu_operations management,edu_mechanical engineering,edu_religious studies,edu_computer systems,edu_ministry,edu_philosophy,edu_civil engineering,edu_rhetoric,edu_logistics,edu_electronics engineering,edu_telecommunications,edu_network administration,edu_criminal justice,edu_law enforcement,edu_statistics,edu_materials science,edu_international business,edu_latin american studies,edu_social science,edu_computer systems engineering,edu_general business,edu_product marketing,edu_spanish,edu_journalism,edu_health science,edu_public health,edu_theological studies,edu_theology,edu_human resources management,edu_social work,edu_supply chain management,edu_chemical engineering,edu_industrial engineering,edu_coaching,edu_fine arts,edu_applied mathematics,edu_comparative literature,edu_aviation,edu_public relations,edu_commerce,edu_theatre,edu_project management,edu_computer science and engineering,edu_piano,edu_piano performance,edu_french,edu_spanish language,edu_general studies,edu_corporate finance,edu_investment banking,edu_chinese language,edu_archeology,edu_classical literature,edu_european history,edu_global studies,edu_public policy,edu_communication studies,edu_applied economics,edu_italian studies,edu_financial management,edu_human relations,edu_information science,edu_music,edu_public administration,edu_government,edu_management science,edu_film,edu_corrections,edu_labor relations,edu_financial economics,edu_neuroscience,edu_premed,edu_geochemistry,edu_geology,edu_legal studies,edu_environmental science,edu_industrial relations,edu_law,edu_financial engineering,edu_finance and marketing,edu_counseling,edu_fashion,edu_aerospace engineering,edu_writing,edu_sustainability,edu_linguistics,edu_general education,edu_physical therapy,edu_elementary education,edu_tourism,edu_business studies,edu_photography,edu_human resource management,edu_biomedical science,edu_kinesiology,edu_forensics,edu_informatics,edu_animation,edu_physiology,edu_multimedia,edu_political science and government,edu_banking,edu_health education,edu_english literature,edu_advertising,edu_creative writing,edu_clinical psychology,edu_latin,edu_chinese,edu_environmental studies,edu_cognitive science,edu_composition,edu_computer programming,edu_actuarial science,edu_fundraising,edu_global health,edu_theater arts,edu_urban studies,edu_social sciences,edu_studio arts,edu_exercise science,edu_european studies,edu_web development,edu_mechatronics engineering,edu_industrial design,edu_television,edu_italian,edu_japanese language studies,edu_applied psychology,edu_human development,edu_compute

# Logistic regression for numerical values

In [48]:
def log_reg(feature):
  X, y = np.array(data[feature]).reshape(-1, 1), np.array(data['stakeholder_category'])
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)
  clf = LogisticRegression(random_state=0)
  clf.fit(trainX, trainy)

  score = 0.5
  for i in np.linspace(0,1,21):
    decisions = (clf.predict_proba(testX) >= i).astype(int)
    score_temp = sum(abs(decisions[:,0]-testy))/len(testy)
    if score_temp > score:
      #print(i, 'y',score_temp,score)
      score = score_temp
      thresh = i
  return score, thresh

In [49]:
features = ['uni_rank', 'degree_score', 'month_of_service', 'skills_count', 'company_count']

for feat in features:
  s, t = log_reg(feat)
  print(round(s,2),feat)

0.63 uni_rank
0.6 degree_score
0.51 month_of_service
0.54 skills_count
0.61 company_count


* multivariate

In [103]:
def log_reg():
  X, y = np.array(data[features]).reshape(-1, len(features)), np.array(data['stakeholder_category'])
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)
  clf = LogisticRegression(random_state=0)
  clf.fit(trainX, trainy)

  score = 0.5
  for i in np.linspace(0,1,21):
    decisions = (clf.predict_proba(testX) >= i).astype(int)
    score_temp = sum(abs(decisions[:,0]-testy))/len(testy)
    if score_temp > score:
      #print(i, 'y',score_temp,score)
      score = score_temp
      thresh = i
  return score, thresh

In [104]:
log_reg()

(0.6293279022403259, 0.4)

# Counting for binary values

In [6]:
def count(feature):
  score = sum(abs(np.array(data[feature])-np.array(data['stakeholder_category'])))/len(np.array(data['stakeholder_category']))
  return score

In [7]:
features = ['sub_role_software', 'us_name', 'company_global', 'level_manager']

for feat in features:
  s = count(feat)
  print(round(s,2),feat)

0.55 sub_role_software
0.55 us_name
0.56 company_global
0.57 level_manager


* gender

In [8]:
cnt_m_DM, cnt_f_DM, cnt_m_nDM, cnt_f_nDM = 0, 0, 0, 0
for i in range(len(data['gender'])):
  if data['gender'][i] == 1 and data['stakeholder_category'][i] == 0:
    cnt_m_nDM += 1
  if data['gender'][i] == -1 and data['stakeholder_category'][i] == 0:
    cnt_f_nDM += 1
  if data['gender'][i] == 1 and data['stakeholder_category'][i] == 1:
    cnt_m_DM += 1
  if data['gender'][i] == -1 and data['stakeholder_category'][i] == 1:
    cnt_f_DM += 1

print(cnt_m_DM/(cnt_m_nDM + cnt_m_DM),
      cnt_f_DM/(cnt_f_nDM + cnt_f_DM))

0.4274447949526814 0.5991735537190083


In [9]:
(cnt_f_DM + cnt_f_nDM)/(cnt_m_DM + cnt_m_nDM + cnt_f_DM + cnt_f_nDM)

0.2762557077625571

In [ ]:
cnt_f_DM + cnt_f_nDM

242

In [ ]:
cnt_m_DM + cnt_m_nDM

634

# SVM

In [96]:
def svm_reg(feature, C, deg):
  X, y = np.array(data[feature]).reshape(-1, 1), np.array(data['stakeholder_category'])
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)
  clf = make_pipeline(
      StandardScaler(), 
      SVC(gamma='auto',
          C=C,
          degree = deg,
          decision_function_shape = 'ovr'
          ))
  clf.fit(trainX, trainy)

  return clf.score(testX, testy)

* each feature

In [101]:
features = ['uni_rank', 'degree_score', 'month_of_service', 'skills_count', 'company_count']

for feat in features:
  score = 0
  for c in range(1, 40):
    print(c, score)
    for d in range(40):
      score = max(score, svm_reg(feat, c, d))
  print(feat, round(score,3),feat)

1 0
2 0.38085539714867617
3 0.38085539714867617
4 0.3910386965376782
5 0.39307535641547864
6 0.395112016293279
7 0.395112016293279
8 0.395112016293279
9 0.395112016293279
10 0.395112016293279
11 0.395112016293279
12 0.395112016293279
13 0.395112016293279
14 0.395112016293279
15 0.395112016293279
16 0.395112016293279
17 0.395112016293279
18 0.395112016293279
19 0.395112016293279
20 0.395112016293279
21 0.395112016293279
22 0.395112016293279
23 0.395112016293279
24 0.395112016293279
25 0.395112016293279
26 0.395112016293279
27 0.395112016293279
28 0.395112016293279
29 0.395112016293279
30 0.395112016293279
31 0.395112016293279
32 0.395112016293279
33 0.395112016293279
34 0.395112016293279
35 0.395112016293279
36 0.395112016293279
37 0.395112016293279
38 0.395112016293279
39 0.395112016293279
uni_rank 0.395 uni_rank
1 0
2 0.395112016293279
3 0.395112016293279
4 0.395112016293279
5 0.395112016293279
6 0.395112016293279
7 0.395112016293279
8 0.395112016293279
9 0.395112016293279
10 0.395112

* multivariable

In [87]:
def svm_reg_mult(C, deg):
  X, y = np.array(data[features]).reshape(-1, len(features)), np.array(data['stakeholder_category'])
  trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.7, random_state=2, stratify=y)
  clf = make_pipeline(
      StandardScaler(), 
      SVC(gamma='auto',
          C=C,
          degree = deg,
          decision_function_shape = 'ovr'
          ))
  clf.fit(trainX, trainy)
  return clf.score(testX, testy)

In [93]:
score = 0
for c in range(1, 40):
  print(c, score)
  for d in range(40):
    score = max(score, svm_reg_mult(c, d))

score

1 0
2 0.5953420669577875
3 0.5953420669577875
4 0.5953420669577875
5 0.5997088791848617
6 0.6055312954876274
7 0.6055312954876274
8 0.6055312954876274
9 0.6055312954876274
10 0.6055312954876274
11 0.6055312954876274
12 0.6055312954876274
13 0.6055312954876274
14 0.6055312954876274
15 0.6055312954876274
16 0.6055312954876274
17 0.6055312954876274
18 0.6055312954876274
19 0.6055312954876274
20 0.6055312954876274
21 0.6055312954876274
22 0.6055312954876274
23 0.6055312954876274
24 0.6055312954876274
25 0.6055312954876274
26 0.6055312954876274
27 0.6055312954876274
28 0.6055312954876274
29 0.6055312954876274
30 0.6055312954876274
31 0.6055312954876274
32 0.6055312954876274
33 0.6055312954876274
34 0.6055312954876274
35 0.6055312954876274
36 0.6055312954876274
37 0.6055312954876274
38 0.6055312954876274
39 0.6055312954876274


0.6055312954876274